In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'/home/dhanushka/Documents/kidneyDiseaseClassification'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path
    